In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#%matplolib inline

In [32]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train['source'] = 'train'
test['source']='test'
data = pd.concat([train,test],ignore_index = True)

C:\Users\muralish\Miniconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [33]:
data.apply(lambda x: sum(x.isnull()))

Age                                0
City_Category                      0
Gender                             0
Marital_Status                     0
Occupation                         0
Product_Category_1                 0
Product_Category_2            245982
Product_Category_3            545809
Product_ID                         0
Purchase                      233599
Stay_In_Current_City_Years         0
User_ID                            0
source                             0
dtype: int64

1.Which age group bought more?

2.Which City people bought more?

3.Which Gender bought more?

4.Married or unmarried?

5.Which product was the most sold?

6.Which user bought the most?

# Functions

### Binning Age

In [34]:
def binning_age(data,method):
    if method =='kids/elder':
        data['Age']=data['Age'].replace({'0-17':'Kids/Elders','51-55':'Kids/Elders','55+':'Kids/Elders'})
    return data
data = binning_age(data,'kids/elder')
data.head()

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID,source
0,Kids/Elders,A,F,0,10,3,NaN,NaN,P00069042,8370.0,2,1000001,train
1,Kids/Elders,A,F,0,10,1,6.0,14.0,P00248942,15200.0,2,1000001,train
2,Kids/Elders,A,F,0,10,12,NaN,NaN,P00087842,1422.0,2,1000001,train
3,Kids/Elders,A,F,0,10,12,14.0,NaN,P00085442,1057.0,2,1000001,train
4,Kids/Elders,C,M,0,16,8,NaN,NaN,P00285442,7969.0,4+,1000002,train


### Filling null

In [35]:
#Fill null
data['Product_Category_2'] = data['Product_Category_2'].fillna(0)
data['Product_Category_3'] = data['Product_Category_3'].fillna(0)
data['Product_Category_2'] = data['Product_Category_2'].astype(int)
data['Product_Category_3'] = data['Product_Category_3'].astype(int)
data.head()

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID,source
0,Kids/Elders,A,F,0,10,3,0,0,P00069042,8370.0,2,1000001,train
1,Kids/Elders,A,F,0,10,1,6,14,P00248942,15200.0,2,1000001,train
2,Kids/Elders,A,F,0,10,12,0,0,P00087842,1422.0,2,1000001,train
3,Kids/Elders,A,F,0,10,12,14,0,P00085442,1057.0,2,1000001,train
4,Kids/Elders,C,M,0,16,8,0,0,P00285442,7969.0,4+,1000002,train


### Stay in current years to numerical

In [36]:
data['Stay_In_Current_City_Years']=data['Stay_In_Current_City_Years'].replace({'4+':4})
data['Stay_In_Current_City_Years'] = data['Stay_In_Current_City_Years'].astype(int)
data.head()

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID,source
0,Kids/Elders,A,F,0,10,3,0,0,P00069042,8370.0,2,1000001,train
1,Kids/Elders,A,F,0,10,1,6,14,P00248942,15200.0,2,1000001,train
2,Kids/Elders,A,F,0,10,12,0,0,P00087842,1422.0,2,1000001,train
3,Kids/Elders,A,F,0,10,12,14,0,P00085442,1057.0,2,1000001,train
4,Kids/Elders,C,M,0,16,8,0,0,P00285442,7969.0,4,1000002,train


In [37]:
train = data[data['source']=='train']
test = data[data['source']=='test']
train = train.drop('source',axis=1)
test = test.drop('source',axis=1)
test = test.drop('Purchase',axis=1)

### Mean Product ID

In [38]:
table = pd.pivot_table(train, values = 'Purchase', columns = 'Product_ID')
table['-1'] = 0
train['mean_purchase_product_ID']=0
train['mean_purchase_product_ID'] = train['Product_ID'].apply(lambda x: table[x])

temp = test.Product_ID.apply(lambda x: x  in table)
ID_not_in_train = test[temp ==False].Product_ID.values

test['Product_ID_copy'] = test['Product_ID']
test.loc[temp == False,'Product_ID']= '-1'

test['mean_purchase_product_ID']=0.0
test['mean_purchase_product_ID'] = test['mean_purchase_product_ID'].astype(float)

test['mean_purchase_product_ID'] = test['Product_ID'].apply(lambda x: table[x]) 

test = test.drop('Product_ID',axis=1)
test = test.rename({'Product_ID_copy':'Product_ID'}, axis='columns')

### Mean User Purchase

In [39]:
table_user = pd.pivot_table(train, values = 'Purchase', columns = 'User_ID')

In [40]:
train['mean_purchase_user_ID']=0
train['mean_purchase_user_ID'] = train['User_ID'].apply(lambda x: table_user[x])

test['mean_purchase_user_ID']=0
test['mean_purchase_user_ID'] = test['User_ID'].apply(lambda x: table_user[x])

### Encoding and dummy variables

In [41]:
train = train.drop('Product_ID',axis=1)
train = train.drop('User_ID',axis=1)

test = test.drop('Product_ID',axis=1)
test = test.drop('User_ID',axis=1)

In [42]:
to_label_encode = ['Age','City_Category','Gender']
to_one_hot_encode = ['Age','City_Category','Product_Category_1','Product_Category_2','Product_Category_3']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in to_label_encode:
    train[i] = le.fit_transform(train[i])

In [43]:
train.to_csv('train-before one hot encoding.csv', encoding='utf-8', index=False)

In [44]:
train = pd.get_dummies(train, columns=to_one_hot_encode, drop_first=True)

In [45]:
#Encoding, dummy variable of test data 
to_label_encode = ['Age','City_Category','Gender']
to_one_hot_encode = ['Age','City_Category','Product_Category_1','Product_Category_2','Product_Category_3']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in to_label_encode:
    test[i] = le.fit_transform(test[i])
test = pd.get_dummies(test, columns=to_one_hot_encode, drop_first=True)
test['Product_Category_1_19']=0
test['Product_Category_1_20']=0

In [46]:
x = train.drop('Purchase',axis=1)
y = train['Purchase']

In [47]:
x.to_csv('x.csv', encoding='utf-8', index=False)
y.loc[-1]= 0.00
y.sort_index()
y.to_csv('y.csv', encoding='utf-8', index=False)
test.to_csv('test-final.csv', encoding='utf-8', index=False)
#train.to_csv('train-final.csv', encoding='utf-8', index=False)